# 1.labelme标注的json格式数据转换为yolo格式

In [22]:
import os
import json

In [18]:
# 要转换的json文件
json_file = '../example_images/labelme_json.json'
# 定义映射关系(目标类别和索引之间的关系)
class_mapping = {
    "crack":0
}

In [20]:
def convert_json_to_yolo(json_file, class_mapping):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    yolo_labels = []

    # 图像的总高度
    img_h = data['imageHeight']
    # 图像的总宽度
    img_w = data['imageWidth']

    # 遍历各个边界框
    for shape in data['shapes']:
        label = shape['label']
        points = shape['points']

        x_min = min(x[0] for x in points)
        x_max = max(x[0] for x in points)
        y_min = min(x[1] for x in points)
        y_max = max(x[1] for x in points)

        class_id = class_mapping.get(label)
        if class_id is not None:
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min

            # 归一化
            width /= img_w
            height /= img_h

            x_center /= img_w
            y_center /= img_h

            yolo_label = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_labels.append(yolo_label)
    return yolo_labels

In [21]:
yolo_labels = convert_json_to_yolo(json_file, class_mapping)

In [28]:
# 保存到yolo格式文件
yolo_filename = os.path.basename(json_file)[:-4] + 'txt'
yolo_filename = os.path.join(os.path.dirname(json_file), yolo_filename)
with open(yolo_filename, 'w') as f:
    for label in yolo_labels:
        f.write(label + '\n')